In [1]:
import numpy as np
import itertools

Q = np.array([
        [2,1],
        [1,2]
    ])

In [2]:
def gradient_interpolator(ksi_bold):
    """B Matrix"""
    ksi, eta = ksi_bold[0], ksi_bold[1]
    grad_phi1 = [-0.25*(1-eta), -0.25*(1-ksi)]
    grad_phi2 = [0.25*(1-eta), -0.25*(1+ksi)]
    grad_phi3 = [0.25*(1+eta), 0.25*(1+ksi)]
    grad_phi4 = [-0.25*(1+eta), 0.25*(1-ksi)]

    return np.array([
        grad_phi1,
        grad_phi2,
        grad_phi3,
        grad_phi4
    ])

jacobian = lambda x, y: x.dot(y)
gamma = lambda j: np.linalg.inv(j)
det_J = lambda j: np.linalg.det(j)

gauss_points = [[-np.sqrt(3)/3,-np.sqrt(3)/3],
                [np.sqrt(3)/3, -np.sqrt(3)/3],
                [np.sqrt(3)/3, np.sqrt(3)/3],
                [-np.sqrt(3)/3, np.sqrt(3)/3]]

phi_1 = lambda ksi, eta: 0.25*(1-ksi)*(1-eta)
phi_2 = lambda ksi, eta: 0.25*(1+ksi)*(1-eta)
phi_3 = lambda ksi, eta: 0.25*(1+ksi)*(1+eta)
phi_4 = lambda ksi, eta: 0.25*(1-ksi)*(1+eta)

phis = np.array([
    phi_1,
    phi_2,
    phi_3,
    phi_4
])

def build_local(M, Q, Q_ab, f_on_nodes, prescribed_vals, flow_vals, glob=False):
    from itertools import product
    
    tipos, flow = flow_vals
    
    if type(tipos) == int:
        t = list()
        t.append(tipos)
        tipos = t
    
    F = np.zeros((4,4))
    
    if len(tipos) > 0:
        tipos = np.array([tipos])
        tipos = tipos.reshape(tipos.shape[1],1)
        for tipo in tipos:
            
            if tipo == 1:
                flow_matrix = np.zeros((4,4))
                print('tipo I')
                eta = -1
                pg = np.sqrt(3)/3
                for x in product([0,1], repeat=2):
                    i, j = x

                    if glob:
                        if i != 0 and j != 0:
                            flow_matrix[i, j] = phis[i](-pg,eta)*phis[j](-pg,eta) +\
                             phis[i](pg, eta)*phis[j](pg, eta)
                    else:
                        flow_matrix[i, j] = phis[i](-pg,eta)*phis[j](-pg,eta) +\
                             phis[i](pg, eta)*phis[j](pg, eta)
                F += flow_matrix

            elif tipo == 2:
                #######################################
                flow_matrix = np.zeros((4,4))
                print('tipo II')
                ksi = 1
                pg = np.sqrt(3)/3
                for x in product([1,2], repeat=2):
                    i, j = x
                    if glob:
                        if j != 0 and i != 0:
                            flow_matrix[i, j] = phis[i](ksi,-pg)*phis[j](ksi, -pg) +\
                             phis[i](ksi, pg)*phis[j](ksi, pg)
                    else:
                        flow_matrix[i, j] = phis[i](ksi, -pg)*phis[j](ksi, -pg) +\
                             phis[i](ksi, pg)*phis[j](ksi, pg)
                F += flow_matrix
                #######################################
                
            elif tipo == 3:
                #######################################
                flow_matrix = np.zeros((4,4))
                print('tipo III')
                eta = 1
                pg = np.sqrt(3)/3
                for x in product([2,3], repeat=2):
                    i, j = x
                    if glob:
                        if i != 0 and j != 0:
                            flow_matrix[i, j] = phis[i](-pg,eta)*phis[j](-pg,eta) +\
                             phis[i](pg, eta)*phis[j](pg, eta)
                    else:
                        flow_matrix[i, j] = phis[i](-pg,eta)*phis[j](-pg,eta) +\
                             phis[i](pg, eta)*phis[j](pg, eta)
                F += flow_matrix
                #######################################
                
            else:
                flow_matrix = np.zeros((4,4))
                print('tipo IV')
                ksi = -1
                pg = np.sqrt(3)/3
                for x in product([3,0], repeat=2):
                    i, j = x
                    if glob:
                        if j != 0 and i != 0:
                            flow_matrix[i, j] = phis[i](ksi,-pg)*phis[j](ksi, -pg) +\
                             phis[i](ksi, pg)*phis[j](ksi, pg)
                    else:
                        flow_matrix[i, j] = phis[i](ksi, -pg)*phis[j](ksi, -pg) +\
                             phis[i](ksi, pg)*phis[j](ksi, pg)
                F += flow_matrix
    
    #################################
    print(F)
    #################################
    
    K_e = np.zeros((4,4))
    for p in gauss_points:
        B = gradient_interpolator(p)
        J = jacobian(M, B)
        G = gamma(J)
        D = det_J(J)
        K_e += (B @ G.T @ Q @ G @ B.T) * D

    F_e = (Q_ab @ f_on_nodes) * D - (K_e @ prescribed_vals) + (F @ flow) * 0.125
    return K_e, F_e

## Criação da discretização 2D, numeração dos nós e numeração dos elementos

In [3]:
verbose = False

In [4]:
nx = 5
ny = 5
X, Y = np.meshgrid(np.linspace(0,1,nx), np.linspace(0,1,ny))

In [5]:
print(X.shape, Y.shape)

(5, 5) (5, 5)


In [6]:
Z = np.array(list(zip(X.ravel(), Y.ravel())))
print(Z.shape)

(25, 2)


In [7]:
nodes = list(enumerate(Z))
nodes_n = nodes[-1][0] + 1
print(nodes_n)

25


In [8]:
elements_n = (nx-1)*(ny-1)
print(elements_n)

16


In [9]:
G = np.zeros((4,elements_n), dtype=np.int64)

In [10]:
num = 0

for d in range(ny-1):
    for i in range((nx-1)):
        G[:,num] = np.array([i+d*nx, i+1+d*nx, i+nx+1+d*nx, i+nx+d*nx])
        num+=1

if verbose:
    for x in range(G.shape[1]):
        print(G[:,x])

## Montagem da matriz rigidez local, e posterior mapeamento para a matriz rigidez global

In [11]:
def map_nodes_to_physical(G, elem, verbose=False):
    n_ = G[:,elem]
    l = list()
    
    for idx in n_:
        l.append(nodes[int(idx)][1])
        if verbose:
            print('global node {0} --> point {1} on R2'.format(idx, nodes[int(idx)][1]))
        
    M = np.array(l).T
    return M

In [12]:
def check(permutations):
    l = []
    t_ = {
        (0, 1): 1,
        (1, 2): 2,
        (2, 3): 3,
        (3, 0): 4
}
    for y in permutations:
        if y in t_.keys():
            l.append(t_[y])
    return l

In [13]:
def l2g(EQ_vector, L2G_matrix, Q_ab, f, prescribed, flow, verbose=False):
    nodes_per_elem, elem_n = L2G_matrix.shape
    
    if verbose:
        print(nodes_per_elem, elem_n)
    
    eq_n = int(np.max(EQ_vector) + 1)
    
    if verbose:
        print(eq_n)
    
    K = np.zeros((eq_n, eq_n))
    F = np.zeros((eq_n, ))
    
    for elem in [15]:
        print('elem {0}'.format(elem))
        global_nodes = L2G_matrix[:,elem]
        f_on_nodes = f[global_nodes]
        pres_on_nodes = prescribed[global_nodes]
        flow_on_nodes = flow[global_nodes]
        f = np.where(np.absolute(flow_on_nodes) > 1e-13)[0]
        perm = itertools.permutations(f, 2)
        tipos = check(perm)
        print(tipos)
        
        if verbose:
            print('global nodes --> {0}'.format(global_nodes))
            print('f on nodes --> {0}'.format(f_on_nodes))
            print('prescribed on nodes --> {0}'.format(pres_on_nodes))
            print('flow on nodes --> {0}'.format(flow_on_nodes))
            
        c = itertools.product(range(nodes_per_elem),repeat=2)
        M = map_nodes_to_physical(L2G_matrix, elem, verbose=verbose)
        glob = True if 0 in global_nodes else False
        local_k, local_F = build_local(M, Q, Q_ab, f_on_nodes, pres_on_nodes, (tipos[0], flow_on_nodes), glob=glob)
        print(local_F)
        
        for x in c:
            aux1 = L2G_matrix[x[0]][elem]
            aux2 = L2G_matrix[x[1]][elem]    
            i = EQ_vector[aux1]
            j = EQ_vector[aux2]
            
            if verbose:
                print('el_node {0} --> glob_node {1} --> eq_vec {2}'.format((x), (aux1, aux2), (i, j)))
                
            if i != -1 and j != -1:
                K[i][j] += local_k[x[0]][x[1]]
                
                if verbose:
                    print('\t----> accumulating to global')
        
        for node in global_nodes:
            i = EQ_vector[node]
            
            if verbose:
                print('node {0} --> equation {1}'.format(node, i))
                
            if i != -1:
                
                if verbose:
                    print('\tnode {0} --> pos {1} on local_f'.format(node, np.where(global_nodes == node)[0]))
                
                F[i] += local_F[np.where(global_nodes == node)[0]]
        
        if verbose:
            print('+'*60)

    return K, F

In [14]:
# phi_1 = lambda ksi, eta: 0.25*(1-ksi)*(1-eta)
# phi_2 = lambda ksi, eta: 0.25*(1+ksi)*(1-eta)
# phi_3 = lambda ksi, eta: 0.25*(1+ksi)*(1+eta)
# phi_4 = lambda ksi, eta: 0.25*(1-ksi)*(1+eta)

# gauss_points = [[-np.sqrt(3)/3,-np.sqrt(3)/3],
#                 [np.sqrt(3)/3, -np.sqrt(3)/3],
#                 [np.sqrt(3)/3, np.sqrt(3)/3],
#                 [-np.sqrt(3)/3, np.sqrt(3)/3]]

interp = np.array([phi_1, phi_2, phi_3, phi_4])

Q_ab = np.zeros((4,4))

verbose = False
for i in range(4):
    for j in range(4):
        for pg in gauss_points:
            Q_ab[i, j] += interp[i](pg[0], pg[1])*interp[j](pg[0], pg[1])
            
            if verbose:
                print('({0}, {1}) --> pg = ({2}, {3})'.format(i, j, pg[0], pg[1]))
        
        if verbose:
            print('+'*60)

In [15]:
f_ = lambda x: (2*np.pi**2)*(2*np.sin(np.pi*x[0])*np.cos(np.pi*x[1]) + np.cos(np.pi*x[0])*np.sin(np.pi*x[1]))
f = np.zeros((nodes_n,))
for pts in nodes:
    f[pts[0]] = f_(pts[1])

In [16]:
prescribed = np.zeros((nodes_n, ))

flow = np.zeros((nodes_n, ))
zero = np.array([0])
# union = zero

gamma_1 = np.where(Z[:,1] == 0)[0]
# union = np.union1d(union, gamma_1)
union = gamma_1
print("gamma 1 {0}".format(gamma_1))
# prescribed[gamma_1] = np.sin(np.pi*Z[gamma_1,0])
flow[gamma_1] = -np.pi*np.cos(np.pi*Z[gamma_1, 0])

gamma_3 = np.setdiff1d(np.where(Z[:,1] == 1)[0], union)
union = np.union1d(union, gamma_3)
print("gamma 3 {0}".format(gamma_3))
flow[gamma_3] = -np.pi*np.cos(np.pi*Z[gamma_3, 0])
# flow[gamma_1] = -np.pi*np.cos(np.pi*Z[gamma_1, 0])

gamma_2 = np.setdiff1d(np.where(Z[:,0] == 1)[0], union)
union = np.union1d(union, gamma_2)
print("gamma 2 {0}".format(gamma_2))
flow[gamma_2] = -2*np.pi*np.cos(np.pi*Z[gamma_2, 1])
# flow[gamma_4] = -2*np.pi*np.cos(np.pi*Z[gamma_4, 1])

# prescribed[gamma_3] = -np.sin(np.pi*Z[gamma_3,0])

gamma_4 = np.setdiff1d(np.where(Z[:,0] == 0)[0], union)
print("gamma 4 {0}".format(gamma_4))
flow[gamma_4] = -2*np.pi*np.cos(np.pi*Z[gamma_4, 1])

flow_nodes = np.concatenate((gamma_1, gamma_2, gamma_3, gamma_4),axis=0)
print('flow nodes {0}'.format(flow_nodes))

d_sols = np.setdiff1d(np.arange(nodes_n), zero)
print('d_sols {0}'.format(d_sols))

gamma 1 [0 1 2 3 4]
gamma 3 [20 21 22 23 24]
gamma 2 [ 9 14 19]
gamma 4 [ 5 10 15]
flow nodes [ 0  1  2  3  4  9 14 19 20 21 22 23 24  5 10 15]
d_sols [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24]


In [17]:
flow

array([ -3.14159265e+00,  -2.22144147e+00,  -1.92367069e-16,
         2.22144147e+00,   3.14159265e+00,  -4.44288294e+00,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
        -4.44288294e+00,  -3.84734139e-16,   0.00000000e+00,
         0.00000000e+00,   0.00000000e+00,  -3.84734139e-16,
         4.44288294e+00,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00,   4.44288294e+00,  -3.14159265e+00,
        -2.22144147e+00,  -1.92367069e-16,   2.22144147e+00,
         3.14159265e+00])

In [18]:
print(flow[gamma_2])
print(flow[gamma_3])

[ -4.44288294e+00  -3.84734139e-16   4.44288294e+00]
[ -3.14159265e+00  -2.22144147e+00  -1.92367069e-16   2.22144147e+00
   3.14159265e+00]


In [19]:
EQ_vector = np.zeros((nodes_n,), dtype=np.int64) - 1
count = 0

for idx in range(EQ_vector.shape[0]):
    if idx in d_sols:
        EQ_vector[idx] = count
        count += 1

In [20]:
K, F = l2g(EQ_vector, G, Q_ab, f, prescribed, flow, verbose=True)

4 16
24
elem 15
[2, 3]
global nodes --> [18 19 24 23]
f on nodes --> [ -2.96088132e+01  -1.39577284e+01  -7.25206766e-15  -2.79154568e+01]
prescribed on nodes --> [ 0.  0.  0.  0.]
flow on nodes --> [ 0.          4.44288294  3.14159265  2.22144147]
global node 18 --> point [ 0.75  0.75] on R2
global node 19 --> point [ 1.    0.75] on R2
global node 24 --> point [ 1.  1.] on R2
global node 23 --> point [ 0.75  1.  ] on R2
tipo II
[[ 0.          0.          0.          0.        ]
 [ 0.          0.66666667  0.33333333  0.        ]
 [ 0.          0.33333333  0.66666667  0.        ]
 [ 0.          0.          0.          0.        ]]
[-0.35100976  0.25293856  0.25012232 -0.32089789]
el_node (0, 0) --> glob_node (18, 18) --> eq_vec (17, 17)
	----> accumulating to global
el_node (0, 1) --> glob_node (18, 19) --> eq_vec (17, 18)
	----> accumulating to global
el_node (0, 2) --> glob_node (18, 24) --> eq_vec (17, 23)
	----> accumulating to global
el_node (0, 3) --> glob_node (18, 23) --> eq_vec

In [21]:
# d = np.linalg.solve(K, F)

In [22]:
# sols = prescribed
# sols[d_sols] = d

In [23]:
# import matplotlib.pyplot as plt
# from mpl_toolkits.mplot3d import Axes3D
# fig = plt.figure(figsize=(14, 14))
# ax = fig.add_subplot(111, projection='3d')

# X, Y = Z[:,0], Z[:,1]

# ax.plot_trisurf(X, Y, sols)
# ax.view_init(20,35)
# plt.show()